    1º mirar si un juego se ha vendido o no (hacer columna en tabala de juegos (1 vendido,0 no vendido))
    2º Agrupar por género los que se han vendido (contarlos) y dado su precio intentar predecir si se venderá
    otro tipo...
    3º Tener en cuenta el descuento aplicado en el caso de que hay tenido descuento (pocos casos)

In [290]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
 
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["multidimensional"]

compra = mycol.find({"movimiento.tipo_movimiento":"compra"})
venta = mycol.find({"movimiento.tipo_movimiento":"venta"})
dfc = pd.DataFrame(list(compra))
dfv = pd.DataFrame(list(venta))
 

## Compras

### Corrección de celdas nulas por celdas con valores nulos

In [291]:
for col in dfc:
    for i, row_value in dfc[col].iteritems():
        if col == "dlc" and dfc[col].isna()[i]:
            dfc[col][i]={'puntero_dlc': -1, 'tipo': 'vacio', 'precio_dlc': -1}
        if col == "juego" and dfc[col].isna()[i]:
            dfc[col][i]={'puntero_juego': -1, 'genero': [], 'precio_juego': -1, 'clasificacion':'vacio'}
dfc.to_csv(r'export_dataframe.csv', sep=',',index = None, header=True)

In [292]:
dfc

,_id,dlc,fecha,juego,movimiento,proveedor
0,5e24a88037d0e43e4c87d958,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2020', 'mes': '01', 'dia': '17'}","{'puntero_juego': 5601, 'genero': ['RPG', 'fre...","{'precio_final': 0, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 16, 'pais': 'Egypt', 'va..."
1,5e24a88037d0e43e4c87d959,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '02', 'dia': '05'}","{'puntero_juego': 7066, 'genero': ['adventure'...","{'precio_final': 5, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 14, 'pais': 'Virgin Isla..."
2,5e24a88037d0e43e4c87d95a,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '06', 'dia': '07'}","{'puntero_juego': 5826, 'genero': [], 'precio_...","{'precio_final': 8, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 34, 'pais': 'Niue', 'val..."
3,5e24a88037d0e43e4c87d95b,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '12', 'dia': '31'}","{'puntero_juego': 7931, 'genero': ['strategy',...","{'precio_final': 20, 'tipo_movimiento': 'compr...","{'puntero_proveedor': 21, 'pais': 'Israel', 'v..."
4,5e24a88037d0e43e4c87d95c,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '01', 'dia': '23'}","{'puntero_juego': 9477, 'genero': ['racing'], ...","{'precio_final': 8, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 29, 'pais': 'New Zealand..."
5,5e24a88037d0e43e4c87d95d,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '09', 'dia': '29'}","{'puntero_juego': 10894, 'genero': ['adventure...","{'precio_final': 7, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 20, 'pais': 'Sierra Leon..."
6,5e24a88037d0e43e4c87d95e,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '02', 'dia': '04'}","{'puntero_juego': 639, 'genero': ['strategy', ...","{'precio_final': 8, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 26, 'pais': 'Laos', 'val..."
7,5e24a88037d0e43e4c87d95f,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '03', 'dia': '04'}","{'puntero_juego': 8611, 'genero': ['action'], ...","{'precio_final': 4, 'tipo_movimiento': 'compra...","{'puntero_proveedor': 30, 'pais': 'Eritrea', '..."
8,5e24a88137d0e43e4c87d960,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '07', 'dia': '07'}","{'puntero_juego': 3042, 'genero': ['action'], ...","{'precio_final': 16, 'tipo_movimiento': 'compr...","{'puntero_proveedor': 6, 'pais': 'Saint Helena..."
9,5e24a88137d0e43e4c87d961,"{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_d...","{'anyo': '2019', 'mes': '11', 'dia': '19'}","{'puntero_juego': 2073, 'genero': [], 'precio_...","{'precio_final': 80, 'tipo_movimiento': 'compr...","{'puntero_proveedor': 9, 'pais': 'Honduras', '..."


## <span style="color:Blue">Desenrollamos el contenido de los documentos embebidos</span>

In [293]:
dfcdlc = pd.DataFrame(list(dfc.dlc))
#añadimos las columnas correspondientes con un dlc y a su vez el correspondiente contenido
dfc=dfc.assign(puntero_dlc = dfcdlc.puntero_dlc) 
dfc=dfc.assign(precio_dlc = dfcdlc.precio_dlc) 
dfc=dfc.assign(tipo_dlc = dfcdlc.tipo) 


dfcjuego = pd.DataFrame(list(dfc.juego))
dfc=dfc.assign(clasificacion_juego = dfcjuego.clasificacion) 
dfc=dfc.assign(genero_juego = dfcjuego.genero) 
dfc=dfc.assign(precio_juego = dfcjuego.precio_juego) 
dfc=dfc.assign(puntero_juego = dfcjuego.puntero_juego) 

dfcmovimiento = pd.DataFrame(list(dfc.movimiento))
dfc=dfc.assign(cantidad_movimiento = dfcmovimiento.cantidad) 
dfc=dfc.assign(precio_final_movimiento = dfcmovimiento.precio_final) 
dfc=dfc.assign(tipo_movimiento = dfcmovimiento.tipo_movimiento) 

dfcfecha = pd.DataFrame(list(dfc.fecha))
dfc=dfc.assign(anyo = dfcfecha.anyo) 
dfc=dfc.assign(mes = dfcfecha.mes) 
dfc=dfc.assign(dia = dfcfecha.dia)

dfcproveedor = pd.DataFrame(list(dfc.proveedor))
dfc=dfc.assign(pais_proveedor = dfcproveedor.pais) 
dfc=dfc.assign(puntero_proveedor = dfcproveedor.puntero_proveedor) 
dfc=dfc.assign(valoracion = dfcproveedor.valoracion)

dfc=dfc.drop(['proveedor', 'fecha','movimiento','juego','dlc'], axis=1)
dfc.to_csv(r'export_dataframeNew.csv', sep=',',index = None, header=True)

# Ventas

In [294]:
for col in dfv:
    for i, row_value in dfv[col].iteritems():
        if col == "dlc" and dfv[col].isna()[i]:
            dfv[col][i]={'puntero_dlc': -1, 'tipo': 'vacio', 'precio_dlc': -1}
        if col == "juego" and dfv[col].isna()[i]:
            dfv[col][i]={'puntero_juego': -1, 'genero': [], 'precio_juego': -1, 'clasificacion':'vacio'}#CAMBIAR PRECIO A PRECIO_JUEGO
dfv.dlc[0]

{'puntero_dlc': -1, 'tipo': 'vacio', 'precio_dlc': -1}

## <span style="color:Blue">Desenrollamos el contenido de los documentos embebidos</span>

In [295]:
#dlc
dfvdlc = pd.DataFrame(list(dfv.dlc))
dfv=dfv.assign(puntero_dlc = dfvdlc.puntero_dlc) 
dfv=dfv.assign(precio_dlc = dfvdlc.precio_dlc) 
dfv=dfv.assign(tipo_dlc = dfvdlc.tipo) 
#juego
dfvjuego = pd.DataFrame(list(dfv.juego))
dfv=dfv.assign(clasificacion_juego = dfvjuego.clasificacion) 
dfv=dfv.assign(genero_juego = dfvjuego.genero) 
dfv=dfv.assign(precio_juego = dfvjuego.precio_juego) 
dfv=dfv.assign(puntero_juego = dfvjuego.puntero_juego) 
#movimiento
dfvmovimiento = pd.DataFrame(list(dfv.movimiento))
dfv=dfv.assign(cantidad_movimiento = dfvmovimiento.cantidad) 
dfv=dfv.assign(precio_final_movimiento = dfvmovimiento.precio_final) 
dfv=dfv.assign(tipo_movimiento = dfvmovimiento.tipo_movimiento)
#fecha
dfvfecha = pd.DataFrame(list(dfv.fecha))
dfv=dfv.assign(anyo = dfvfecha.anyo) 
dfv=dfv.assign(mes = dfvfecha.mes) 
dfv=dfv.assign(dia = dfvfecha.dia)
#ubicacion
dfvubicacion = pd.DataFrame(list(dfv.ubicacion))
dfv=dfv.assign(pais_ubicacion = dfvubicacion.pais) 
dfv=dfv.assign(ciudad_ubicacion = dfvubicacion.ciudad) 
#usuario
dfvusuario = pd.DataFrame(list(dfv.usuario))
dfv=dfv.assign(puntero_usuario = dfvusuario.puntero_usuario) 
dfv=dfv.assign(sexo_usuario = dfvusuario.sexo)
dfv=dfv.assign(fechaNacimiento_usuario = dfvusuario.fechaNacimiento) 
dfv=dfv.assign(pais_usuario = dfvusuario.pais) 
dfv=dfv.assign(ciudad_usuario = dfvusuario.ciudad) 

dfv=dfv.drop(['descuento','ubicacion','fecha','usuario','juego','dlc','movimiento'], axis=1)
dfv.to_csv(r'export_dataframeNewVentas.csv', sep=',',index = None, header=True)
dfv

,_id,puntero_dlc,precio_dlc,tipo_dlc,clasificacion_juego,genero_juego,precio_juego,puntero_juego,cantidad_movimiento,precio_final_movimiento,...,anyo,mes,dia,pais_ubicacion,ciudad_ubicacion,puntero_usuario,sexo_usuario,fechaNacimiento_usuario,pais_usuario,ciudad_usuario
0,5e24a9ad37d0e43df0bf1514,-1,-1,vacio,AAA,"[adventure, simulation]",15,2117,1,15,...,2019,06,03,Spain,Alacant,1,H,1999-09-13,Spain,Alacant
1,5e24a9ad37d0e43df0bf1515,-1,-1,vacio,AAA,[RPG],3,6063,1,3,...,2019,07,07,Spain,Ceuta,2,M,1988-07-13,Spain,Ceuta
2,5e24a9ad37d0e43df0bf1516,-1,-1,vacio,AAA,[RPG],21,3993,1,21,...,2017,11,08,Spain,Ceuta,2,M,1988-07-13,Spain,Ceuta
3,5e24a9ad37d0e43df0bf1517,-1,-1,vacio,INDIE,"[adventure, casual]",15,11392,1,15,...,2019,12,25,Spain,Ceuta,2,M,1988-07-13,Spain,Ceuta
4,5e24a9ad37d0e43df0bf1518,-1,-1,vacio,INDIE,"[action, adventure, RPG]",5,10240,1,5,...,2017,10,22,Spain,Santa Coloma de Gramenet,3,M,1994-07-24,Spain,Santa Coloma de Gramenet
5,5e24a9ad37d0e43df0bf1519,-1,-1,vacio,AAA,"[action, adventure]",7,5951,1,7,...,2017,05,17,Spain,Palma de Mallorca,4,H,1999-06-30,Spain,Palma de Mallorca
6,5e24a9ad37d0e43df0bf151a,-1,-1,vacio,INDIE,"[casual, strategy, simulation]",0,2699,1,0,...,2019,08,10,Spain,Palma de Mallorca,4,H,1999-06-30,Spain,Palma de Mallorca
7,5e24a9ad37d0e43df0bf151b,-1,-1,vacio,INDIE,[],0,6519,1,0,...,2020,01,11,Spain,Palma de Mallorca,4,H,1999-06-30,Spain,Palma de Mallorca
8,5e24a9ad37d0e43df0bf151c,-1,-1,vacio,INDIE,"[action, RPG]",13,12202,1,13,...,2017,08,14,Spain,Oviedo,5,H,2000-10-04,Spain,Oviedo
9,5e24a9ad37d0e43df0bf151d,-1,-1,vacio,AAA,[casual],25,1498,1,25,...,2017,03,06,Spain,Logroño,6,H,1997-02-23,Spain,Logroño


## <span style="color:Blue">Sacamos la edad del usuario y borramos columnas innecesarias</span>

In [296]:
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
array=[]
for fila in dfv.iterrows():
    fe=fila[1].fechaNacimiento_usuario
    fecha = datetime.strptime(fe, '%Y-%m-%d')
    hoy = datetime.now()
    age = relativedelta(hoy, fecha)
    array.append(age.years)
dfv=dfv.assign(edad = array)
datos=dfv.drop(['_id','puntero_dlc','precio_dlc','tipo_dlc','genero_juego','precio_juego','puntero_juego',
                'cantidad_movimiento','tipo_movimiento','anyo','mes','dia',
               'pais_ubicacion','ciudad_ubicacion','puntero_usuario','fechaNacimiento_usuario','ciudad_usuario'], axis=1)


In [297]:
datos=datos.loc[lambda datos: datos['clasificacion_juego'] != "vacio"]
datos

,clasificacion_juego,precio_final_movimiento,sexo_usuario,pais_usuario,edad
0,AAA,15,H,Spain,20
1,AAA,3,M,Spain,31
2,AAA,21,M,Spain,31
3,INDIE,15,M,Spain,31
4,INDIE,5,M,Spain,25
5,AAA,7,H,Spain,20
6,INDIE,0,H,Spain,20
7,INDIE,0,H,Spain,20
8,INDIE,13,H,Spain,19
9,AAA,25,H,Spain,22


### <span style="color:Blue">Conversión a valores numéricos</span>

In [298]:
#A continuacion prodecemos a convertir los campos no numéricos a numéricos
#datos=datos.loc[datos['clasificacion_juego'] =="AAA"] = 0
datos.loc[datos.clasificacion_juego =="AAA",'clasificacion_juego']=0
datos.loc[datos.clasificacion_juego =="INDIE",'clasificacion_juego']=1

datos.loc[datos.sexo_usuario =="H",'sexo_usuario']=0
datos.loc[datos.sexo_usuario =="M",'sexo_usuario']=1

datos.loc[datos.pais_usuario =="United States",'pais_usuario']=0
datos.loc[datos.pais_usuario =="Austria",'pais_usuario']=1
datos.loc[datos.pais_usuario =="Belgium",'pais_usuario']=2
datos.loc[datos.pais_usuario =="France",'pais_usuario']=3
datos.loc[datos.pais_usuario =="Russian Federation",'pais_usuario']=4
datos.loc[datos.pais_usuario =="Spain",'pais_usuario']=5
datos.loc[datos.pais_usuario =="United Kingdom",'pais_usuario']=6

datos= datos.rename(columns={'clasificacion_juego': 'label'})

datos

,label,precio_final_movimiento,sexo_usuario,pais_usuario,edad
0,0,15,0,5,20
1,0,3,1,5,31
2,0,21,1,5,31
3,1,15,1,5,31
4,1,5,1,5,25
5,0,7,0,5,20
6,1,0,0,5,20
7,1,0,0,5,20
8,1,13,0,5,19
9,0,25,0,5,22


# CLASIFICACIÓN

In [299]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os     

import statsmodels.api as sm

from IPython.display import display, Markdown
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression

from statsmodels.tools.eval_measures import rmse # Para calcular el error

from math import sqrt
import scipy.cluster.hierarchy as hac

%matplotlib inline
%config IPCompleter.greedy=True
warnings.filterwarnings('ignore')

# Preparamos el dataset de entrenamiento y el de test

In [300]:
# entrenamiento y test
train_features, test_features = train_test_split(datos, test_size=0.35)

train_labels = train_features['label']
test_labels = test_features['label']
train_features =train_features.drop('label', axis=1)
test_features =test_features.drop('label', axis=1)
columnas = list(train_features)

### <span style="color:Green">Gradient Boosting</span>

In [301]:
#Gradient Boosting

gbc = GradientBoostingClassifier(loss='deviance', random_state=0)
gbc.fit(train_features, train_labels)
prediccion = gbc.predict(test_features)
report = classification_report(test_labels, prediccion)
rms = sqrt(mean_squared_error(test_labels, prediccion))
print("Correct classification rate:", accuracy_score(test_labels, prediccion))

print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.6790540540540541
              precision    recall  f1-score   support

           0       0.68      0.50      0.57       258
           1       0.68      0.82      0.74       334

    accuracy                           0.68       592
   macro avg       0.68      0.66      0.66       592
weighted avg       0.68      0.68      0.67       592



,tn,fp,fn,tp
0,128,130,60,274


### <span style="color:Green">Logistic Regresion</span>

In [302]:
#Logistic Regresion
import sys
np.set_printoptions(threshold=sys.maxsize)

modelo = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.5, fit_intercept=True, intercept_scaling=1, class_weight='balanced', random_state=None, 
                            solver='lbfgs', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=3)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
proba =  modelo.predict_proba(test_features)

prediccion = np.round(prediccion)
prediccion= pd.DataFrame(prediccion)


report = classification_report(test_labels, prediccion.values)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))

print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
cm


Correct classification rate: 0.6165540540540541
              precision    recall  f1-score   support

           0       0.56      0.60      0.58       258
           1       0.67      0.63      0.65       334

    accuracy                           0.62       592
   macro avg       0.61      0.61      0.61       592
weighted avg       0.62      0.62      0.62       592



,tn,fp,fn,tp
0,154,104,123,211


### <span style="color:Green">Random Forest Regressor</span>

In [303]:

#Random Forest Regressor
modelo = RandomForestRegressor(n_estimators=350, oob_score = True, random_state=0, n_jobs=3)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
prediccion = np.round(prediccion)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))

print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)


Correct classification rate: 0.6114864864864865
              precision    recall  f1-score   support

           0       0.56      0.53      0.54       258
           1       0.65      0.68      0.66       334

    accuracy                           0.61       592
   macro avg       0.60      0.60      0.60       592
weighted avg       0.61      0.61      0.61       592



,tn,fp,fn,tp
0,136,122,108,226


### <span style="color:Green">Random Forest clasificator</span>

In [311]:

#RamdomForest clasificador
modelo = RandomForestClassifier(criterion='gini', random_state=0)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))

print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.606418918918919
              precision    recall  f1-score   support

           0       0.55      0.53      0.54       258
           1       0.65      0.66      0.65       334

    accuracy                           0.61       592
   macro avg       0.60      0.60      0.60       592
weighted avg       0.61      0.61      0.61       592



,tn,fp,fn,tp
0,138,120,113,221


### <span style="color:Green">Gaussian Naive Bayes</span>

In [305]:
from sklearn.naive_bayes import GaussianNB
modelo= GaussianNB()
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

Correct classification rate: 0.6807432432432432
              precision    recall  f1-score   support

           0       0.86      0.32      0.46       258
           1       0.65      0.96      0.77       334

    accuracy                           0.68       592
   macro avg       0.75      0.64      0.62       592
weighted avg       0.74      0.68      0.64       592



,tn,fp,fn,tp
0,82,176,13,321


### <span style="color:Green">Bernoulli Naive Bayes</span>

In [306]:
from sklearn.naive_bayes import BernoulliNB
modelo = BernoulliNB()
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)


Correct classification rate: 0.5743243243243243
              precision    recall  f1-score   support

           0       0.54      0.17      0.25       258
           1       0.58      0.89      0.70       334

    accuracy                           0.57       592
   macro avg       0.56      0.53      0.48       592
weighted avg       0.56      0.57      0.51       592



,tn,fp,fn,tp
0,43,215,37,297


### <span style="color:Green">DecisionTreeClassifier</span>

In [307]:
from sklearn import tree
modelo = tree.DecisionTreeClassifier(random_state=0)
modelo.fit(train_features, train_labels)
prediccion = modelo.predict(test_features)
report = classification_report(test_labels, prediccion)
print("Correct classification rate:", accuracy_score(test_labels, prediccion))
print(report)
cm = confusion_matrix(test_labels, prediccion).ravel()
cm = pd.DataFrame(cm.reshape((1,4)), columns=['tn', 'fp', 'fn', 'tp'])
display(cm)

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import collections

import os     

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
dot_data = tree.export_graphviz(modelo,
                                feature_names=columnas,
                                out_file=None,
                                filled=True,
                                rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data)

colors = ('turquoise', 'orange')
edges = collections.defaultdict(list)

for edge in graph.get_edge_list():
    edges[edge.get_source()].append(int(edge.get_destination()))

for edge in edges:
    edges[edge].sort()    
    for i in range(2):
        dest = graph.get_node(str(edges[edge][i]))[0]
        dest.set_fillcolor(colors[i])

graph.write_png('tree.png')

Correct classification rate: 0.5996621621621622
              precision    recall  f1-score   support

           0       0.53      0.63      0.58       258
           1       0.67      0.58      0.62       334

    accuracy                           0.60       592
   macro avg       0.60      0.60      0.60       592
weighted avg       0.61      0.60      0.60       592



,tn,fp,fn,tp
0,162,96,141,193


True